### Import necessary packages

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from nltk.tokenize import word_tokenize
import texthero as hero
import pandas as pd
import numpy as np
import nltk
import time

pd.set_option('display.max_columns', None)

C:\Users\fmagh\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Read and clean data

In [2]:
df_praise = pd.read_csv("data/cleaned-non-quantifier-data.csv")

### Creates day of week column

In [3]:
df_praise["Date"] =  pd.to_datetime(df_praise["Date"], format='%Y-%m-%d')
df_praise['Date'] = df_praise['Date'].dt.day_name()
df_praise.rename({'Date': 'day_of_week'}, axis=1, inplace=True)
df_praise = df_praise.iloc[:,:12]
# df_praise.drop(labels=[range(12,25)], axis=1, inplace=True)
df_praise[(df_praise['IH per Praise'] -  48.829077) < 1]
df_praise

,To,From,Reason for dishing,Server,day_of_week,Room,v1 norm,v2 norm,v3 norm,Avg %,IH per Praise,IH per person
0,zeptimusQ,Tam2140,for hosting this kicking params party!,Token Engineering Commons,Friday,🙏praise,10000.0,100.0,200.0,0.001963,2.355252,39.04072
1,zeptimusQ,iviangita,for hosting and leading a lot of params parties,Token Engineering Commons,Friday,🙏praise,10000.0,100.0,100.0,0.001663,1.995539,39.04072
2,zeptimusQ,JuankBell,for testing and deploying the bot to record an...,Token Engineering Commons,Wednesday,🙏praise,1000.0,200.0,200.0,0.001342,1.610872,39.04072
3,zeptimusQ,iviangita,for the huge success of the MVV process,Token Engineering Commons,Friday,🙏praise,1000.0,200.0,100.0,0.001043,1.251160,39.04072
4,zeptimusQ,iviangita,"for his awesome work on the recorder bot, for ...",Token Engineering Commons,Friday,🙏praise,1000.0,200.0,100.0,0.001043,1.251160,39.04072
...,...,...,...,...,...,...,...,...,...,...,...,...
7620,atacas,liviade,their presence and great contributions to the ...,Telegram,Tuesday,TE Praise,30.0,60.0,40.0,0.001534,NaN,NaN
7621,Luukweber,iviangita,participating in the Praisegiving Call 😀,Telegram,Saturday,TE Praise,40.0,30.0,30.0,0.001180,NaN,NaN
7622,krrisis,GriffGreen,sharing the gitcoin blog post... lots of great...,Telegram,Wednesday,Commons Stack,10.0,40.0,40.0,0.001103,NaN,NaN
7623,sbilbao,JessicaZartler,joining the TE Praise channel and the TEC lab ...,Telegram,Friday,TE Praise,50.0,30.0,10.0,0.001005,NaN,NaN


In [4]:
for index_col in [2, 5]:
    df_praise.iloc[:,index_col] = hero.clean(df_praise.iloc[:,index_col])

cleaned_data_praise = df_praise.iloc[:,2]

C:\Users\fmagh\anaconda3\lib\site-packages\texthero\preprocessing.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(r"^\d+\s|\s\d+\s|\s\d+$", " ")
C:\Users\fmagh\anaconda3\lib\site-packages\texthero\preprocessing.py:81: FutureWarning: The default value of regex will change from True to False in a future version.
  return input.str.replace(pat, '')


In [5]:
#tokenize and tag the card text
praise_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(cleaned_data_praise)]
#display the tagged docs
praise_docs[:10]

[TaggedDocument(words=['hosting', 'kicking', 'params', 'party'], tags=[0]),
 TaggedDocument(words=['hosting', 'leading', 'lot', 'params', 'parties'], tags=[1]),
 TaggedDocument(words=['testing', 'deploying', 'bot', 'record', 'display', 'meetings'], tags=[2]),
 TaggedDocument(words=['huge', 'success', 'mvv', 'process'], tags=[3]),
 TaggedDocument(words=['awesome', 'work', 'recorder', 'bot', 'help', 'documentation', 'always', 'getting', 'recordings', 'going', 'pushing', 'transparency', 'wg', 'good', 'personality'], tags=[4]),
 TaggedDocument(words=['taking', 'care', 'ivy', 'access', 'twitter', 'dms'], tags=[5]),
 TaggedDocument(words=['many', 'community', 'members', 'user', 'testing', 'last', 'month', 'slowly', 'surely', 'make', 'project', 'something', 'proud', 'release'], tags=[6]),
 TaggedDocument(words=['keeping', 'stewards', 'working', 'groups', 'transparent', 'stewards', 'audit', 'forum'], tags=[7]),
 TaggedDocument(words=['awesome', 'memes'], tags=[8]),
 TaggedDocument(words=['atte

In [6]:
#instantiate model
model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 40)
#build vocab
model.build_vocab(praise_docs)
#train model
model.train(praise_docs, total_examples=model.corpus_count
            , epochs=model.epochs)

In [7]:
#generate vectors
card2vec = [model.infer_vector((cleaned_data_praise[i].split(' '))) 
            for i in range(0,len(cleaned_data_praise))]

In [8]:
#Create a list of lists
dtv= np.array(card2vec).tolist()
#set list to dataframe column
df_praise['Reason for dishing'] = dtv
df_praise.head(2)

,To,From,Reason for dishing,Server,day_of_week,Room,v1 norm,v2 norm,v3 norm,Avg %,IH per Praise,IH per person
0,zeptimusQ,Tam2140,"[0.0780276209115982, -0.15283703804016113, 0.0...",Token Engineering Commons,Friday,praise,10000.0,100.0,200.0,0.001963,2.355252,39.04072
1,zeptimusQ,iviangita,"[0.26589980721473694, -0.20526523888111115, 0....",Token Engineering Commons,Friday,praise,10000.0,100.0,100.0,0.001663,1.995539,39.04072


In [9]:
# imputing null data
for index_ in range(6,12):
    df_praise.iloc[:,index_].fillna(0, inplace=True)

In [10]:
new_cols_order = ['To', 'From', 'Reason for dishing', 'Server', 'Room','v1 norm', 'v2 norm',
                  'v3 norm', 'Avg %', 'IH per Praise','IH per person', 'day_of_week']

In [11]:
df_praise = df_praise.reindex(columns=new_cols_order)
df_praise

,To,From,Reason for dishing,Server,Room,v1 norm,v2 norm,v3 norm,Avg %,IH per Praise,IH per person,day_of_week
0,zeptimusQ,Tam2140,"[0.0780276209115982, -0.15283703804016113, 0.0...",Token Engineering Commons,praise,10000.0,100.0,200.0,0.001963,2.355252,39.04072,Friday
1,zeptimusQ,iviangita,"[0.26589980721473694, -0.20526523888111115, 0....",Token Engineering Commons,praise,10000.0,100.0,100.0,0.001663,1.995539,39.04072,Friday
2,zeptimusQ,JuankBell,"[0.34356874227523804, -0.016960332170128822, 0...",Token Engineering Commons,praise,1000.0,200.0,200.0,0.001342,1.610872,39.04072,Wednesday
3,zeptimusQ,iviangita,"[0.0823730081319809, 0.014336182735860348, -0....",Token Engineering Commons,praise,1000.0,200.0,100.0,0.001043,1.251160,39.04072,Friday
4,zeptimusQ,iviangita,"[0.5389704704284668, 0.3260210454463959, 0.396...",Token Engineering Commons,praise,1000.0,200.0,100.0,0.001043,1.251160,39.04072,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...
7620,atacas,liviade,"[0.04043954610824585, -0.0379733145236969, 0.0...",Telegram,te praise,30.0,60.0,40.0,0.001534,0.000000,0.00000,Tuesday
7621,Luukweber,iviangita,"[0.1699880212545395, -0.04136759042739868, 0.1...",Telegram,te praise,40.0,30.0,30.0,0.001180,0.000000,0.00000,Saturday
7622,krrisis,GriffGreen,"[0.22629161179065704, -0.2741315960884094, 0.0...",Telegram,commons stack,10.0,40.0,40.0,0.001103,0.000000,0.00000,Wednesday
7623,sbilbao,JessicaZartler,"[-0.3024839460849762, 0.003090156242251396, 0....",Telegram,te praise,50.0,30.0,10.0,0.001005,0.000000,0.00000,Friday


In [12]:
df_final = pd.DataFrame(df_praise.index)
start_time = time.time()
for index_ in [0, 1, 3, 4]:
    x = df_praise.iloc[:,index_].values.reshape(-1, 1)
    y = OneHotEncoder().fit_transform(x).toarray()
    column_name = df_praise.columns[index_]
    df_praise.iloc[:,index_] = pd.DataFrame({column_name: list(y)})
    df_final = pd.concat([df_final, df_praise[column_name].apply(pd.Series)], axis=1)

df_final = pd.concat([df_final, df_praise['Reason for dishing'].apply(pd.Series)], axis=1)   
print(f"total runtime is: {time.time() - start_time}")
df_final.iloc[:,0] = df_praise['day_of_week']
df_final

total runtime is: 7.247981548309326


,0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,Friday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078028,-0.152837,0.074841,0.054083,-0.008514,-0.039565,0.049173,-0.135190,-0.194626,-0.013404,0.027590,0.006353,-0.251044,0.052610,-0.036122,-0.010427,0.034613,0.157694,0.113862,-0.123388,0.033830,-0.023565,0.110471,-0.049576,-0.059287,0.191410,-0.102111,-0.053691,0.083630,-0.187672,0.081146,0.139691,0.028917,0.162697,-0.049961,0.166420,0.144602,-0.186654,0.169942,0.091894,0.139823,-0.075241,-0.308053,0.048243,0.133557,0.040235,0.055218,0.071769,-0.145108,0.024745,-0.111979,0.008657,-0.004695,-0.112967,0.116479,0.180266,-0.043979,-0.063169,0.062023,-0

### Export vectorized data for furture usage

In [13]:
df_final.to_csv("outputs/vectorized_praise.csv")